一个Web服务器对请求进行响应的典型步骤如下。

1. 服务器对客户端发来的请求进行解析
2. 请求被转发给一个预定义的处理器
3. 处理其可能会从数据库中取数据
4. 处理器根据取出的数据对模版进行渲染
5. 处理器向客户返回渲染后的内容作为对请求的响应

## Rails cookie

```ruby
# Sets a simple session cookie.
# This cookie will be deleted when the user's browser is closed.

cookies[:user_name] = "david"

# Cookie values are String based. Other data types need to be serialized.
cookies[:lat_lon] = JSON.generate([47.68, -122.37])

# Sets a cookie that expires in 1 hour.
cookies[:login] = { value: "XJ-122", expires: 1.hour.from_now }

# Sets a signed cookie, which prevents users from tampering with its value.
# The cookie is signed by your app's `secrets.secret_key_base` value.
# It can be read using the signed method `cookies.signed[:name]`
cookies.signed[:user_id] = 33

# Sets an encrypted cookie value before sending it to the client which
# prevent users from reading and tampering with its value.
# The cookie is signed by your app's `secrets.secret_key_base` value.
# It can be read using the encrypted method `cookies.encrypted[:name]`
cookies.encrypted[:discount] = 45

# Sets a "permanent" cookie (which expires in 20 years from now).
cookies.permanent[:login] = "XJ-122"

# You can also chain these methods:
cookies.permanent.signed[:login] = "XJ-122"
```

![](https://img.vim-cn.com/b5/b6ade4b93e3ef201331b5fab9333593f43801d.jpg)

## 签名cookie

通常会存储用户名，可能还有用户Id、用户最后一次成功登陆的时间，以及网站觉得有用的任何信息。除了用户的相关信息之外，签名`cookie`还包含一个签名，服务器可以使用这个签名来验证浏览器发送的信息是否经过改动

验证`cookie`所需的一切信息都存储在`cookie`里面，`cookie`可以包含额外的信息，并且对这些签名也很容易

坏处：正确的处理这些签名很难。很容易忘记对数据进行签名，或者忘记验证数据的签名。从而造成安全漏洞。

## 令牌cookie

令牌`cookie`会在`cookie`里面存储一串随机字节作为令牌，服务器可以根据令牌在数据库中查找令牌的拥有者。随着时间推移，旧令牌会被新令牌取代

添加信息非常容易。`cookie`的提及非常小，因此移动终端和速度较慢的客户端可以更快的发送请求。

坏处：需要在服务器中存储更多信息。如果使用的是关系数据库，那么载入和存储`cookie`的代价可能会很高


除了用户登陆信息之外，`Fake Web Retailer`还可以将用户的访问时长和已浏览商品的数量等信息存储到数据库里面，这样便于将来通过分析这些信息来学习如何更好地向用户推销商品

一般来说，用户在决定购买某个或某些商品之前，通常都会先浏览多个不同的商品，而记录用户浏览过的所有商品以及用户最后一次访问页面的时间等信息来，通常会导致大量的数据库写入。从长远来看，用户的这些浏览数据的确非常有用，但问题在于，即使经过优化，大多数关系数据库每秒也只能插入、更新或者删除`200~2000`个数据行。尽管批量插入、批量更新和批量删除等操作可以更快的速度执行，但因为客户端每次浏览网页都只更新少数几个行，所以告诉的批量插入在这里并不适用。

```python

def check_token(conn, token)
  return conn.get('login:', token)
```

用户每次浏览页面的时候的时候，程序都会对用户存储在登陆散列里面的信息进行更新，并将用户的令牌和当前时间戳添加到记录最近登陆用户的有序集合里面，并在被记录的商品数量超过25个时，对这个有序集合进行裁剪。

```python
def update_token(conn, token, user, item = None):
  timestamp = time.time()
  conn.hset('login:', token, user)
  conn.zadd('recent', token, timestamp)
  if item:
    conn.zadd('viewed:' + token, item, timestamp)
    conn.zremrangebyrank('viewed:' + token, 0, -26)
```

```redis
127.0.0.1:6379> ZADD myzset 1 "one"
(integer) 1
127.0.0.1:6379> ZADD myzset 2 "two"
(integer) 1
127.0.0.1:6379> ZADD myzset 3 "three"
(integer) 1
127.0.0.1:6379> ZREMRANGEBYRANK myzset 0 1
(integer) 2
127.0.0.1:6379> ZRANGE myzset 0 -1 WITHSCORES
1) "three"
2) "3"
```

```python
QUIT = FALSE
LIMIT = 10000000

def clean_sessions(conn):
    while not QUIT:
        size = conn.zcard('recent')
        if size <= LIMIT:
            time.sleep(1)
            continue
        end_index = min(size - LIMIT, 100)
        tokens = conn.zrange('recent:', 0, end_index - 1)
        
        session_keys = []
        for token in tokens:
            session_keys.append('viewed' + token)
        
        conn.delete(*session_keys)
        conn.hdel('login:', *tokens)
        conn.zrem('recent', *tokens)
        
```

```redis

127.0.0.1:6379> ZADD salary 2000 tom
(integer) 1
127.0.0.1:6379> ZCARD salary
(integer) 1
127.0.0.1:6379> ZADD salary 5000 jack
(integer) 1
127.0.0.1:6379> ZCARD salary
(integer) 2
127.0.0.1:6379> EXISTS non_exists_key
(integer) 0
127.0.0.1:6379> ZCARD non_exists_key
(integer) 0
```

因为存储会话所需的内存会随着时间的推移而不断增加，所以我们要定期清理旧的会话数据。清理旧会话的程序由一个循环构成，这个循环每次执行的时候，都会检查存储最近登录令牌的有序集合的大小，如果有序集合的大小超过了限制，那么程序就会从有序集合里面一处醉倒的100个最旧的令牌，并从记录用户登录信息的散列里面，移除被删除令牌对应的用户的信息，并对存储了这些用户的最近浏览记录的有序集合进行清理。与此相反，如果令牌的数量未超过限制，那么程序会先休眠一秒，之后再重新进行检查

熟悉多线程编程的或者并发条件的读者可能会发现代码清单的清理函数实际上包含一个竞争条件：如果清理函数正在删除某个用户的信息，而这个用户又在同一时间访问文件的话，那么竞争条件就会导致用户的信息被错误的删除

## 使用Redis实现购物车

使用`cookie`实现购物车，也就是将整个购物车都存储到`cookie`里面的做法很常见，这种做法的一大优点是无须对数据库进行写入就可以实现购物车功能，而缺点则是程序需要重新解析和验证`cookie`，确保`cookie`格式正确，并且包含的商品都是真正可以买的商品。`cookie`购物车还有一个缺点：因为浏览器发送请求都会连`cookie`一起发送，所以如果购物车`cookie`体积比较大，那么请求和处理的速度可能会降低。

购物车的定义非常简单：每个用户的购物车都是一个散列，这个散列存储了商品订购数量之间的映射。

```python
def add_to_cart(conn, session, item, count):
    if count <= 0:
        conn.hrem('cart:' + session, item)
    else:
        conn.hset('cart:' + session, item, count)
```